In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.optim as optim
import numpy as np
import torch.nn.functional as F



In [ ]:


transform = transforms.Compose([

    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),     
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

## Creat the Dataset and Dataloader 

In [ ]:

train_dataset = datasets.FashionMNIST(root='./data', train = True, transform= transform, download= True)
test_dataset = datasets.FashionMNIST(root = './data', train = False, transform = transform, download = True)


In [ ]:

batch_size = 64
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size)


## Creat Model

In [ ]:
class model(nn.Module):
  def __init__(self, drop_out_rate = 0.5):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels= 1, out_channels= 32,kernel_size= 2 , stride = 2, padding = 1)  # what is the padding here to get to this model
    self.norm1 = nn.BatchNorm2d(32)
    self.con2 = nn.Conv2d(in_channels = 32, out_channels = 62, kernel_size= 2, stride = 2, padding = 1)
    self.norm2 = nn.BatchNorm2d(64)e = 3, stride = 2)
    self.drop = nn.Dropout(p = drop_out
    self.pool = nn.MaxPool2d(kernel_siz_rate)
    self.fc1 = nn.Linear(62 * 3 * 3, 64)  # just give the input of the linear so get this to them to creat some thing


    self.dropout_fc = nn.Dropout(p = 0.4)
    self.fc2 = nn.Linear(64, 10)


  def forward(self, x):
    x = self.pool(F.relu(self.norm1(self.conv1(x))))
    x = self.pool(F.relu(self.norm2(self.Conv2d(x))))
    x = self.drop(x)
    batch_size, channels, hieght, weight = x.shape


    
    x = x.view(-1, 128 * 4 * 4)
    x = F.relu(self.fc1(x))
    x = self.dropout_fc(x)
    out = self.fc2(x)


    return out



In [24]:
learning_rate = 0.001
num_epoch = 12
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
model = model().to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Trainig Loop :

In [25]:
def train(model, test_loader, device , num_epoch):

  all_step = len(train_loader)

  all_loss = []
  for epoch in range(num_epoch):
    model.train()
    epoch_loss = []

    for i, (images , labels) in enumerate(test_loader):
      image = images.to(device)
      label = labels.to(device)

      out = model(image)
      _, prediction = torch.max(out, 1)

      loss = loss(label, prediction)
      loss.append(loss.item())

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i+1)%200 == 0:
        print(f'Epoch : [{epoch}/{num_epoch}], Step : [{i+1}/{all_step}], Loss : {loss.item():.4f}%')

    all_loss.extend(epoch_loss)



  return model, all_loss






## Testing Model :

In [ ]:
def test(model , test_loader, device):
  model.eval()
  n_correct = 0
  n_sample = 0

  with torch.no_grad():

    for images, labels in test_loader:
      image, label = images.to(device),labels.to(device)

      out = model(image)
      _, predictio = torch.max(out.data, 1)

      n_sample += label.size(0)
      n_correct += (predictio == label).sum().item()

      accuracy = 100.0 * n_correct/n_sample

      return accuracy

# call your method her to run all of that I make the sapce for tath here to them to make this to them :


print(f"Acccuracy on test set is : {accuracy :.2f}%")  # give the accuracu from the call of the model



In [ ]:
torch.save(model.state_dict(), 'fashion_cnn.pth')